In [1]:
%%capture
%run Data_Cleaning.ipynb

In [2]:
random_seed = 42
training_size = 0.85
validation_size = 0.1

# Get unique month periods
month_periods = sorted(three_ff_cleaned_df['Date'].dt.to_period('M').unique())
# Split size calculation
total_periods = three_ff_cleaned_df['Date'].dt.to_period('M').nunique()
train_size = int(total_periods * training_size)
val_size = int(total_periods * validation_size)
test_size = total_periods - train_size - val_size

# print(month_periods)
print("train_size is :", train_size, "months")
print("val_size is :", val_size, "months")
print("test_size is :", test_size, "months")

# ElasticNet
l1_ratios = [0.1, 0.5, 0.9]

# SVR
svr_C_values = [0.1, 1, 10]
svr_epsilons = [0.01, 0.1]

# XGboost
xgb_params = [(50, 0.1), (100, 0.05), (200, 0.01)]  # (n_estimators, learning_rate))

# DecisionTreeRegressor
tree_depths = [2, 4, 6, 8, 10]

print("##################################")
print("####### Hyperparameters ##########")
print("For Ridge: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]")
print("For Lasso: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]")
print("For ElasticNet: l1_ratios =", l1_ratios)
print("For SVR - svr_C_values:", svr_C_values)
print("For SVR - svr_epsilons:", svr_epsilons)
print("For Xgboost - n_estimators and learning rate", xgb_params)
print("For DecisionTreeRegressor", tree_depths)

train_size is : 182 months
val_size is : 21 months
test_size is : 12 months
##################################
####### Hyperparameters ##########
For Ridge: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
For Lasso: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
For ElasticNet: l1_ratios = [0.1, 0.5, 0.9]
For SVR - svr_C_values: [0.1, 1, 10]
For SVR - svr_epsilons: [0.01, 0.1]
For Xgboost - n_estimators and learning rate [(50, 0.1), (100, 0.05), (200, 0.01)]
For DecisionTreeRegressor [2, 4, 6, 8, 10]


### 5 factor fama french modelling

In [3]:
# Convert all columns except 'Date' to float
five_ff_cleaned_df = five_ff_cleaned_df.sort_values(by='Date').reset_index(drop=True)
five_ff_cleaned_df.head(4)

,Date,AAPL.O,NVDA.O,MSFT.O,AMZN.O,LLY,WMT,XOM,MA,UNH,...,SUNE.O,TCRT.O,WINT.O,DGLY.O,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2007-02-28,-1.315037,1.135388,-9.120327,3.828094,-2.867782,1.291682,-3.320376,-3.996326,-0.153198,...,4.710145,-0.564442,-13.182539,-16.907633,-1.96,1.29,-0.14,-0.51,-0.71,0.38
1,2007-03-31,9.357873,-7.430645,-1.070674,1.647066,2.107322,-2.855538,5.125842,-0.880898,1.502644,...,-1.991531,-3.650742,7.443259,14.090545,0.68,0.20,-0.97,0.64,-0.65,0.43
2,2007-04-30,7.153685,13.348787,7.164454,43.265350,9.613921,2.044975,5.077626,4.993695,0.169763,...,7.561204,4.776457,22.567200,31.585295,3.49,-2.04,-1.45,1.15,1.03,0.44
3,2007-05-31,19.419975,5.181152,2.474475,11.986431,-0.866247,-0.670016,4.664026,29.199315,3.171925,...,-13.074411,-1.125715,13.226797,-0.417537,3.24,0.40,-0.65,1.58,-1.37,0.41


### Hyperparameter Tuning - Ridge Regression

In [4]:
# Define the Fama-French factor columns
ff_5_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in five_ff_cleaned_df.columns if col not in ff_5_factors]

# Start global timer
start_all = time.time()

# ------------------ Initialize storage ------------------
all_ridge_results = []             # List of dicts with best model info per stock
best_ridge_models = {}             # Dict of best Ridge model per stock

# ------------------ Model Selection Loop ------------------
# Loop through each stock
for idx, stock in enumerate(stock_columns):
    print(f"\n🟠 Ridge - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()
    # Subset and process
    df_subset = five_ff_cleaned_df[ff_5_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'Stock_Return', 'RF']].dropna()
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Excess_Return'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']])

    ridge_results = []
    alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

    # loop each hyperparameter for each stock
    for alpha in alphas:

        model = Ridge(alpha=alpha, random_state = random_seed)

        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end_month = month_periods[i - 1]
            val_start_month = month_periods[i]
            val_end_month = month_periods[i + 2]
            
            
#             print("train_end_month")
#             print(train_end_month)
#             print("val_start_month")
#             print(val_start_month)
#             print("val_end_month")
#             print(val_end_month)

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end_month
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start_month) & \
                       (df_subset['Date'].dt.to_period('M') <= val_end_month)

            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)

            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            val_rmse_list.append(rmse)

        avg_val_rmse = np.mean(val_rmse_list)

        ridge_results.append({
            'Stock': stock,
            'Alpha': alpha,
            'Avg_Validation_RMSE': avg_val_rmse,
            'Model': model
        })

    # Sort and pick best model for this stock
    ridge_results = sorted(ridge_results, key=lambda x: x['Avg_Validation_RMSE'])
    best_result = ridge_results[0] #lowest RMSE

    # Store best model and result
    all_ridge_results.append(best_result)
    best_ridge_models[stock] = best_result['Model']
    
    elapsed = time.time() - start_time
    print(f"✅ Finished stock {stock} | Time: {elapsed:.2f}s")

    # End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🟠 Ridge - Stock 1/975: AAPL.O
✅ Finished stock AAPL.O | Time: 0.58s

🟠 Ridge - Stock 2/975: NVDA.O
✅ Finished stock NVDA.O | Time: 0.52s

🟠 Ridge - Stock 3/975: MSFT.O
✅ Finished stock MSFT.O | Time: 0.49s

🟠 Ridge - Stock 4/975: AMZN.O
✅ Finished stock AMZN.O | Time: 0.49s

🟠 Ridge - Stock 5/975: LLY
✅ Finished stock LLY | Time: 0.50s

🟠 Ridge - Stock 6/975: WMT
✅ Finished stock WMT | Time: 0.39s

🟠 Ridge - Stock 7/975: XOM
✅ Finished stock XOM | Time: 0.36s

🟠 Ridge - Stock 8/975: MA
✅ Finished stock MA | Time: 0.38s

🟠 Ridge - Stock 9/975: UNH
✅ Finished stock UNH | Time: 0.38s

🟠 Ridge - Stock 10/975: ORCL.K
✅ Finished stock ORCL.K | Time: 0.37s

🟠 Ridge - Stock 11/975: COST.O
✅ Finished stock COST.O | Time: 0.38s

🟠 Ridge - Stock 12/975: NFLX.O
✅ Finished stock NFLX.O | Time: 0.36s

🟠 Ridge - Stock 13/975: HD
✅ Finished stock HD | Time: 0.39s

🟠 Ridge - Stock 14/975: CVX
✅ Finished stock CVX | Time: 0.37s

🟠 Ridge - Stock 15/975: CRM
✅ Finished stock CRM | Time: 0.39s

🟠 Ridge - 

✅ Finished stock EQT | Time: 0.67s

🟠 Ridge - Stock 124/975: ROK
✅ Finished stock ROK | Time: 0.43s

🟠 Ridge - Stock 125/975: MPWR.O
✅ Finished stock MPWR.O | Time: 0.35s

🟠 Ridge - Stock 126/975: MCHP.O
✅ Finished stock MCHP.O | Time: 0.34s

🟠 Ridge - Stock 127/975: ANSS.O
✅ Finished stock ANSS.O | Time: 0.39s

🟠 Ridge - Stock 128/975: DTE
✅ Finished stock DTE | Time: 0.40s

🟠 Ridge - Stock 129/975: DXCM.O
✅ Finished stock DXCM.O | Time: 0.39s

🟠 Ridge - Stock 130/975: IP
✅ Finished stock IP | Time: 0.41s

🟠 Ridge - Stock 131/975: AEE
✅ Finished stock AEE | Time: 0.36s

🟠 Ridge - Stock 132/975: PPG
✅ Finished stock PPG | Time: 0.37s

🟠 Ridge - Stock 133/975: PPL
✅ Finished stock PPL | Time: 0.36s

🟠 Ridge - Stock 134/975: MTD
✅ Finished stock MTD | Time: 0.36s

🟠 Ridge - Stock 135/975: WBD.O
✅ Finished stock WBD.O | Time: 0.36s

🟠 Ridge - Stock 136/975: TYL
✅ Finished stock TYL | Time: 0.36s

🟠 Ridge - Stock 137/975: EL
✅ Finished stock EL | Time: 0.39s

🟠 Ridge - Stock 138/975: ATO
✅

✅ Finished stock X | Time: 0.47s

🟠 Ridge - Stock 244/975: HSIC.O
✅ Finished stock HSIC.O | Time: 0.53s

🟠 Ridge - Stock 245/975: LNW.O
✅ Finished stock LNW.O | Time: 0.52s

🟠 Ridge - Stock 246/975: CHE
✅ Finished stock CHE | Time: 0.53s

🟠 Ridge - Stock 247/975: CRL
✅ Finished stock CRL | Time: 0.64s

🟠 Ridge - Stock 248/975: DRS.O
✅ Finished stock DRS.O | Time: 0.48s

🟠 Ridge - Stock 249/975: RGEN.O
✅ Finished stock RGEN.O | Time: 0.48s

🟠 Ridge - Stock 250/975: LSCC.O
✅ Finished stock LSCC.O | Time: 0.41s

🟠 Ridge - Stock 251/975: EXAS.O
✅ Finished stock EXAS.O | Time: 0.43s

🟠 Ridge - Stock 252/975: HAS.O
✅ Finished stock HAS.O | Time: 0.40s

🟠 Ridge - Stock 253/975: PARA.O
✅ Finished stock PARA.O | Time: 0.39s

🟠 Ridge - Stock 254/975: DCI
✅ Finished stock DCI | Time: 0.41s

🟠 Ridge - Stock 255/975: CHDN.O
✅ Finished stock CHDN.O | Time: 0.36s

🟠 Ridge - Stock 256/975: MKTX.O
✅ Finished stock MKTX.O | Time: 0.36s

🟠 Ridge - Stock 257/975: NYT
✅ Finished stock NYT | Time: 0.36s

🟠 

✅ Finished stock ANF | Time: 0.40s

🟠 Ridge - Stock 363/975: JWN
✅ Finished stock JWN | Time: 0.38s

🟠 Ridge - Stock 364/975: ITGR.K
✅ Finished stock ITGR.K | Time: 0.38s

🟠 Ridge - Stock 365/975: SPR
✅ Finished stock SPR | Time: 0.53s

🟠 Ridge - Stock 366/975: DORM.O
✅ Finished stock DORM.O | Time: 0.51s

🟠 Ridge - Stock 367/975: ALE
✅ Finished stock ALE | Time: 0.55s

🟠 Ridge - Stock 368/975: MMS
✅ Finished stock MMS | Time: 0.47s

🟠 Ridge - Stock 369/975: FIZZ.O
✅ Finished stock FIZZ.O | Time: 0.53s

🟠 Ridge - Stock 370/975: TDS
✅ Finished stock TDS | Time: 0.39s

🟠 Ridge - Stock 371/975: CBZ
✅ Finished stock CBZ | Time: 0.42s

🟠 Ridge - Stock 372/975: BCO
✅ Finished stock BCO | Time: 0.42s

🟠 Ridge - Stock 373/975: AVNT.K
✅ Finished stock AVNT.K | Time: 0.40s

🟠 Ridge - Stock 374/975: AVAV.O
✅ Finished stock AVAV.O | Time: 0.37s

🟠 Ridge - Stock 375/975: IESC.O
✅ Finished stock IESC.O | Time: 0.39s

🟠 Ridge - Stock 376/975: M
✅ Finished stock M | Time: 0.37s

🟠 Ridge - Stock 377/97

✅ Finished stock STGW.O | Time: 0.37s

🟠 Ridge - Stock 481/975: WKC
✅ Finished stock WKC | Time: 0.35s

🟠 Ridge - Stock 482/975: MNKD.O
✅ Finished stock MNKD.O | Time: 0.44s

🟠 Ridge - Stock 483/975: MCRI.O
✅ Finished stock MCRI.O | Time: 0.51s

🟠 Ridge - Stock 484/975: APLD.O
✅ Finished stock APLD.O | Time: 0.47s

🟠 Ridge - Stock 485/975: SRCE.O
✅ Finished stock SRCE.O | Time: 0.55s

🟠 Ridge - Stock 486/975: OMCL.O
✅ Finished stock OMCL.O | Time: 0.62s

🟠 Ridge - Stock 487/975: INOD.O
✅ Finished stock INOD.O | Time: 0.43s

🟠 Ridge - Stock 488/975: FL
✅ Finished stock FL | Time: 0.37s

🟠 Ridge - Stock 489/975: NTCT.O
✅ Finished stock NTCT.O | Time: 0.36s

🟠 Ridge - Stock 490/975: EPC
✅ Finished stock EPC | Time: 0.37s

🟠 Ridge - Stock 491/975: ATSG.O
✅ Finished stock ATSG.O | Time: 0.35s

🟠 Ridge - Stock 492/975: ADEA.O
✅ Finished stock ADEA.O | Time: 0.36s

🟠 Ridge - Stock 493/975: ROG
✅ Finished stock ROG | Time: 0.34s

🟠 Ridge - Stock 494/975: NHC
✅ Finished stock NHC | Time: 0.40s


✅ Finished stock OPY | Time: 0.38s

🟠 Ridge - Stock 598/975: REPX.K
✅ Finished stock REPX.K | Time: 0.39s

🟠 Ridge - Stock 599/975: CVLG.K
✅ Finished stock CVLG.K | Time: 0.41s

🟠 Ridge - Stock 600/975: OSPN.O
✅ Finished stock OSPN.O | Time: 0.38s

🟠 Ridge - Stock 601/975: FWRD.O
✅ Finished stock FWRD.O | Time: 0.37s

🟠 Ridge - Stock 602/975: SCVL.O
✅ Finished stock SCVL.O | Time: 0.34s

🟠 Ridge - Stock 603/975: JACK.O
✅ Finished stock JACK.O | Time: 0.33s

🟠 Ridge - Stock 604/975: SENEA.O
✅ Finished stock SENEA.O | Time: 0.36s

🟠 Ridge - Stock 605/975: WLDN.O
✅ Finished stock WLDN.O | Time: 0.34s

🟠 Ridge - Stock 606/975: CASS.O
✅ Finished stock CASS.O | Time: 0.39s

🟠 Ridge - Stock 607/975: SMP
✅ Finished stock SMP | Time: 0.40s

🟠 Ridge - Stock 608/975: CTLP.O
✅ Finished stock CTLP.O | Time: 0.43s

🟠 Ridge - Stock 609/975: FARO.O
✅ Finished stock FARO.O | Time: 0.47s

🟠 Ridge - Stock 610/975: EBF
✅ Finished stock EBF | Time: 0.50s

🟠 Ridge - Stock 611/975: USNA.K
✅ Finished stock US

✅ Finished stock RGCO.O | Time: 0.35s

🟠 Ridge - Stock 715/975: TSSI.O
✅ Finished stock TSSI.O | Time: 0.37s

🟠 Ridge - Stock 716/975: LFVN.O
✅ Finished stock LFVN.O | Time: 0.40s

🟠 Ridge - Stock 717/975: ESCA.O
✅ Finished stock ESCA.O | Time: 0.37s

🟠 Ridge - Stock 718/975: SGMO.O
✅ Finished stock SGMO.O | Time: 0.37s

🟠 Ridge - Stock 719/975: MPAA.O
✅ Finished stock MPAA.O | Time: 0.36s

🟠 Ridge - Stock 720/975: DENN.O
✅ Finished stock DENN.O | Time: 0.39s

🟠 Ridge - Stock 721/975: FXNC.O
✅ Finished stock FXNC.O | Time: 0.39s

🟠 Ridge - Stock 722/975: SWKH.O
✅ Finished stock SWKH.O | Time: 0.40s

🟠 Ridge - Stock 723/975: UTMD.O
✅ Finished stock UTMD.O | Time: 0.37s

🟠 Ridge - Stock 724/975: LAKE.O
✅ Finished stock LAKE.O | Time: 0.38s

🟠 Ridge - Stock 725/975: GENC.K
✅ Finished stock GENC.K | Time: 0.37s

🟠 Ridge - Stock 726/975: LTBR.O
✅ Finished stock LTBR.O | Time: 0.42s

🟠 Ridge - Stock 727/975: HQI.O
✅ Finished stock HQI.O | Time: 0.38s

🟠 Ridge - Stock 728/975: STRT.O
✅ Finish

✅ Finished stock FKWL.O | Time: 0.38s

🟠 Ridge - Stock 830/975: DOMH.O
✅ Finished stock DOMH.O | Time: 0.35s

🟠 Ridge - Stock 831/975: DLHC.O
✅ Finished stock DLHC.O | Time: 0.36s

🟠 Ridge - Stock 832/975: PED
✅ Finished stock PED | Time: 0.36s

🟠 Ridge - Stock 833/975: ALTS.O
✅ Finished stock ALTS.O | Time: 0.36s

🟠 Ridge - Stock 834/975: CULP.K
✅ Finished stock CULP.K | Time: 0.36s

🟠 Ridge - Stock 835/975: SUP
✅ Finished stock SUP | Time: 0.34s

🟠 Ridge - Stock 836/975: CATO.K
✅ Finished stock CATO.K | Time: 0.36s

🟠 Ridge - Stock 837/975: CYCC.O
✅ Finished stock CYCC.O | Time: 0.37s

🟠 Ridge - Stock 838/975: ICAD.O
✅ Finished stock ICAD.O | Time: 0.35s

🟠 Ridge - Stock 839/975: WFCF.O
✅ Finished stock WFCF.O | Time: 0.38s

🟠 Ridge - Stock 840/975: LINK.O
✅ Finished stock LINK.O | Time: 0.41s

🟠 Ridge - Stock 841/975: CTSO.O
✅ Finished stock CTSO.O | Time: 0.38s

🟠 Ridge - Stock 842/975: CNVS.O
✅ Finished stock CNVS.O | Time: 0.38s

🟠 Ridge - Stock 843/975: CNTY.O
✅ Finished stock C

✅ Finished stock ARTW.O | Time: 0.37s

🟠 Ridge - Stock 946/975: AIMD.O
✅ Finished stock AIMD.O | Time: 0.37s

🟠 Ridge - Stock 947/975: AIM
✅ Finished stock AIM | Time: 0.35s

🟠 Ridge - Stock 948/975: STRR.O
✅ Finished stock STRR.O | Time: 0.37s

🟠 Ridge - Stock 949/975: SSY
✅ Finished stock SSY | Time: 0.36s

🟠 Ridge - Stock 950/975: GTBP.O
✅ Finished stock GTBP.O | Time: 0.45s

🟠 Ridge - Stock 951/975: SGMA.O
✅ Finished stock SGMA.O | Time: 0.37s

🟠 Ridge - Stock 952/975: OGEN.K
✅ Finished stock OGEN.K | Time: 0.37s

🟠 Ridge - Stock 953/975: RIME.O
✅ Finished stock RIME.O | Time: 0.38s

🟠 Ridge - Stock 954/975: SNGX.O
✅ Finished stock SNGX.O | Time: 0.34s

🟠 Ridge - Stock 955/975: SNOA.O
✅ Finished stock SNOA.O | Time: 0.36s

🟠 Ridge - Stock 956/975: AEMD.O
✅ Finished stock AEMD.O | Time: 0.38s

🟠 Ridge - Stock 957/975: GBR
✅ Finished stock GBR | Time: 0.35s

🟠 Ridge - Stock 958/975: KNW
✅ Finished stock KNW | Time: 0.39s

🟠 Ridge - Stock 959/975: BKYI.O
✅ Finished stock BKYI.O | Time

### Hyperparameter Tuning - Lasso Regression

In [5]:
# Define the Fama-French factor columns
ff_5_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in five_ff_cleaned_df.columns if col not in ff_5_factors]

# Start global timer
start_all = time.time()

# ------------------ Initialize storage ------------------
all_lasso_results = []
best_lasso_models = {}

# ------------------ Model Selection Loop ------------------
# Loop through each stock
for idx, stock in enumerate(stock_columns):
    print(f"\n🟠 Lasso - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()
    # Subset and process
    df_subset = five_ff_cleaned_df[ff_5_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'Stock_Return', 'RF']].dropna()
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Excess_Return'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']])

    lasso_results = []

    # loop each hyperparameter for each stock
    for alpha in alphas:

        model = Lasso(alpha=alpha, random_state=random_seed, max_iter=10000)

        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end_month = month_periods[i - 1]
            val_start_month = month_periods[i]
            val_end_month = month_periods[i + 2]
            
#             print("train_end_month")
#             print(train_end_month)
#             print("val_start_month")
#             print(val_start_month)
#             print("val_end_month")
#             print(val_end_month)

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end_month
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start_month) & \
                       (df_subset['Date'].dt.to_period('M') <= val_end_month)

            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)

            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            val_rmse_list.append(rmse)

        lasso_results.append({
            'Stock': stock,
            'Alpha': alpha,
            'Avg_Validation_RMSE': np.mean(val_rmse_list),
            'Model': model
        })

    lasso_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_lasso_models[stock] = lasso_results[0]['Model']
    all_lasso_results.append(lasso_results[0])

print(f"✅ Lasso done for {stock} | Best alpha: {lasso_results[0]['Alpha']}")

# End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🟠 Lasso - Stock 1/975: AAPL.O

🟠 Lasso - Stock 2/975: NVDA.O

🟠 Lasso - Stock 3/975: MSFT.O

🟠 Lasso - Stock 4/975: AMZN.O

🟠 Lasso - Stock 5/975: LLY

🟠 Lasso - Stock 6/975: WMT

🟠 Lasso - Stock 7/975: XOM

🟠 Lasso - Stock 8/975: MA

🟠 Lasso - Stock 9/975: UNH

🟠 Lasso - Stock 10/975: ORCL.K

🟠 Lasso - Stock 11/975: COST.O

🟠 Lasso - Stock 12/975: NFLX.O

🟠 Lasso - Stock 13/975: HD

🟠 Lasso - Stock 14/975: CVX

🟠 Lasso - Stock 15/975: CRM

🟠 Lasso - Stock 16/975: CSCO.O

🟠 Lasso - Stock 17/975: MRK

🟠 Lasso - Stock 18/975: ABT

🟠 Lasso - Stock 19/975: MCD

🟠 Lasso - Stock 20/975: TMO

🟠 Lasso - Stock 21/975: ISRG.O

🟠 Lasso - Stock 22/975: RTX

🟠 Lasso - Stock 23/975: QCOM.O

🟠 Lasso - Stock 24/975: ADBE.O

🟠 Lasso - Stock 25/975: AMGN.O

🟠 Lasso - Stock 26/975: INTU.O

🟠 Lasso - Stock 27/975: AMD.O

🟠 Lasso - Stock 28/975: CAT

🟠 Lasso - Stock 29/975: TXN.O

🟠 Lasso - Stock 30/975: NEE

🟠 Lasso - Stock 31/975: DHR

🟠 Lasso - Stock 32/975: PFE

🟠 Lasso - Stock 33/975: BLK

🟠 Lasso - 


🟠 Lasso - Stock 260/975: EVR

🟠 Lasso - Stock 261/975: TTEK.O

🟠 Lasso - Stock 262/975: RRX

🟠 Lasso - Stock 263/975: SIRI.O

🟠 Lasso - Stock 264/975: HALO.O

🟠 Lasso - Stock 265/975: GAP

🟠 Lasso - Stock 266/975: EXLS.O

🟠 Lasso - Stock 267/975: ATI

🟠 Lasso - Stock 268/975: TTC

🟠 Lasso - Stock 269/975: ZION.O

🟠 Lasso - Stock 270/975: BIO

🟠 Lasso - Stock 271/975: MHK

🟠 Lasso - Stock 272/975: APA.O

🟠 Lasso - Stock 273/975: CVLT.O

🟠 Lasso - Stock 274/975: AGCO.K

🟠 Lasso - Stock 275/975: BRKR.O

🟠 Lasso - Stock 276/975: RLI

🟠 Lasso - Stock 277/975: IVZ

🟠 Lasso - Stock 278/975: ONTO.K

🟠 Lasso - Stock 279/975: CWST.O

🟠 Lasso - Stock 280/975: QRVO.O

🟠 Lasso - Stock 281/975: AAON.O

🟠 Lasso - Stock 282/975: ALK

🟠 Lasso - Stock 283/975: UFPI.O

🟠 Lasso - Stock 284/975: FLS

🟠 Lasso - Stock 285/975: SNV

🟠 Lasso - Stock 286/975: TGTX.O

🟠 Lasso - Stock 287/975: TFX

🟠 Lasso - Stock 288/975: CELH.O

🟠 Lasso - Stock 289/975: EAT

🟠 Lasso - Stock 290/975: PEGA.O

🟠 Lasso - Stock 291


🟠 Lasso - Stock 511/975: SCS

🟠 Lasso - Stock 512/975: VECO.O

🟠 Lasso - Stock 513/975: INVX.K

🟠 Lasso - Stock 514/975: DCOM.O

🟠 Lasso - Stock 515/975: EIG

🟠 Lasso - Stock 516/975: HLIT.O

🟠 Lasso - Stock 517/975: HLX

🟠 Lasso - Stock 518/975: IDT

🟠 Lasso - Stock 519/975: BLFS.O

🟠 Lasso - Stock 520/975: GIII.O

🟠 Lasso - Stock 521/975: CRAI.O

🟠 Lasso - Stock 522/975: IMKTA.O

🟠 Lasso - Stock 523/975: OPK.O

🟠 Lasso - Stock 524/975: SAFT.O

🟠 Lasso - Stock 525/975: RES

🟠 Lasso - Stock 526/975: UCTT.O

🟠 Lasso - Stock 527/975: GYRE.O

🟠 Lasso - Stock 528/975: MODG.K

🟠 Lasso - Stock 529/975: TILE.O

🟠 Lasso - Stock 530/975: MSEX.O

🟠 Lasso - Stock 531/975: MRTN.O

🟠 Lasso - Stock 532/975: PRG

🟠 Lasso - Stock 533/975: INVA.O

🟠 Lasso - Stock 534/975: DGII.O

🟠 Lasso - Stock 535/975: APOG.O

🟠 Lasso - Stock 536/975: GERN.O

🟠 Lasso - Stock 537/975: CLMT.O

🟠 Lasso - Stock 538/975: BBSI.O

🟠 Lasso - Stock 539/975: WGO

🟠 Lasso - Stock 540/975: AMN

🟠 Lasso - Stock 541/975: AORT.K




🟠 Lasso - Stock 758/975: KEQU.O

🟠 Lasso - Stock 759/975: SAVA.O

🟠 Lasso - Stock 760/975: PESI.O

🟠 Lasso - Stock 761/975: EGAN.O

🟠 Lasso - Stock 762/975: BSET.O

🟠 Lasso - Stock 763/975: LXRX.O

🟠 Lasso - Stock 764/975: QVCGA.O

🟠 Lasso - Stock 765/975: CSBR.O

🟠 Lasso - Stock 766/975: KRMD.O

🟠 Lasso - Stock 767/975: ACNT.O

🟠 Lasso - Stock 768/975: RAIL.O

🟠 Lasso - Stock 769/975: SNCR.O

🟠 Lasso - Stock 770/975: VXRT.O

🟠 Lasso - Stock 771/975: RCMT.O

🟠 Lasso - Stock 772/975: CMT

🟠 Lasso - Stock 773/975: LCTX.K

🟠 Lasso - Stock 774/975: ISSC.O

🟠 Lasso - Stock 775/975: INFU.K

🟠 Lasso - Stock 776/975: AXR

🟠 Lasso - Stock 777/975: IVAC.O

🟠 Lasso - Stock 778/975: PLCE.O

🟠 Lasso - Stock 779/975: LTRX.O

🟠 Lasso - Stock 780/975: PPIH.O

🟠 Lasso - Stock 781/975: GNSS.O

🟠 Lasso - Stock 782/975: GALT.O

🟠 Lasso - Stock 783/975: GAIA.O

🟠 Lasso - Stock 784/975: KVHI.O

🟠 Lasso - Stock 785/975: HURC.O

🟠 Lasso - Stock 786/975: NOTV.O

🟠 Lasso - Stock 787/975: DXLG.O

🟠 Lasso - Stoc

### Hyperparameter Tuning - ElasticNet

In [6]:
# Define the Fama-French factor columns
ff_5_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in five_ff_cleaned_df.columns if col not in ff_5_factors]

# Start global timer
start_all = time.time()

# ------------------ ElasticNet Regression ------------------

all_elastic_results = []
best_elastic_models = {}

# ------------------ Model Selection Loop ------------------
# Loop through each stock
for idx, stock in enumerate(stock_columns):
    print(f"\n🟠 ElasticNet - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()
    # Subset and process
    df_subset = five_ff_cleaned_df[ff_5_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'Stock_Return', 'RF']].dropna()
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Excess_Return'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']])

    elastic_results = []
    for alpha in alphas:
        for l1_ratio in l1_ratios:
            model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=random_seed, max_iter=10000)
            val_rmse_list = []

            for i in range(train_size, train_size + val_size - 3):
                train_end = month_periods[i - 1]
                val_start = month_periods[i]
                val_end = month_periods[i + 2]

                train_mask = df_subset['Date'].dt.to_period('M') <= train_end
                val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & (df_subset['Date'].dt.to_period('M') <= val_end)


                X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
                y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
                X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
                y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

                model.fit(X_train, y_train)
                rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
                val_rmse_list.append(rmse)

            elastic_results.append({
                'Stock': stock,
                'Alpha': alpha,
                'L1_Ratio': l1_ratio,
                'Avg_Validation_RMSE': np.mean(val_rmse_list),
                'Model': model
            })

    elastic_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_elastic_models[stock] = elastic_results[0]['Model']
    all_elastic_results.append(elastic_results[0])

print(f"✅ ElasticNet done for {stock} | Best α: {elastic_results[0]['Alpha']} | L1: {elastic_results[0]['L1_Ratio']}")


🟠 ElasticNet - Stock 1/975: AAPL.O

🟠 ElasticNet - Stock 2/975: NVDA.O

🟠 ElasticNet - Stock 3/975: MSFT.O

🟠 ElasticNet - Stock 4/975: AMZN.O

🟠 ElasticNet - Stock 5/975: LLY

🟠 ElasticNet - Stock 6/975: WMT

🟠 ElasticNet - Stock 7/975: XOM

🟠 ElasticNet - Stock 8/975: MA

🟠 ElasticNet - Stock 9/975: UNH

🟠 ElasticNet - Stock 10/975: ORCL.K

🟠 ElasticNet - Stock 11/975: COST.O

🟠 ElasticNet - Stock 12/975: NFLX.O

🟠 ElasticNet - Stock 13/975: HD

🟠 ElasticNet - Stock 14/975: CVX

🟠 ElasticNet - Stock 15/975: CRM

🟠 ElasticNet - Stock 16/975: CSCO.O

🟠 ElasticNet - Stock 17/975: MRK

🟠 ElasticNet - Stock 18/975: ABT

🟠 ElasticNet - Stock 19/975: MCD

🟠 ElasticNet - Stock 20/975: TMO

🟠 ElasticNet - Stock 21/975: ISRG.O

🟠 ElasticNet - Stock 22/975: RTX

🟠 ElasticNet - Stock 23/975: QCOM.O

🟠 ElasticNet - Stock 24/975: ADBE.O

🟠 ElasticNet - Stock 25/975: AMGN.O

🟠 ElasticNet - Stock 26/975: INTU.O

🟠 ElasticNet - Stock 27/975: AMD.O

🟠 ElasticNet - Stock 28/975: CAT

🟠 ElasticNet - St


🟠 ElasticNet - Stock 225/975: GME

🟠 ElasticNet - Stock 226/975: EXEL.O

🟠 ElasticNet - Stock 227/975: RGLD.O

🟠 ElasticNet - Stock 228/975: COHR.K

🟠 ElasticNet - Stock 229/975: DOX.O

🟠 ElasticNet - Stock 230/975: IPG

🟠 ElasticNet - Stock 231/975: TECH.O

🟠 ElasticNet - Stock 232/975: ATR

🟠 ElasticNet - Stock 233/975: WBA.O

🟠 ElasticNet - Stock 234/975: MTZ

🟠 ElasticNet - Stock 235/975: SRPT.O

🟠 ElasticNet - Stock 236/975: SAIA.O

🟠 ElasticNet - Stock 237/975: SEIC.O

🟠 ElasticNet - Stock 238/975: RRC

🟠 ElasticNet - Stock 239/975: WYNN.O

🟠 ElasticNet - Stock 240/975: CRS

🟠 ElasticNet - Stock 241/975: CIEN.K

🟠 ElasticNet - Stock 242/975: ALB

🟠 ElasticNet - Stock 243/975: X

🟠 ElasticNet - Stock 244/975: HSIC.O

🟠 ElasticNet - Stock 245/975: LNW.O

🟠 ElasticNet - Stock 246/975: CHE

🟠 ElasticNet - Stock 247/975: CRL

🟠 ElasticNet - Stock 248/975: DRS.O

🟠 ElasticNet - Stock 249/975: RGEN.O

🟠 ElasticNet - Stock 250/975: LSCC.O

🟠 ElasticNet - Stock 251/975: EXAS.O

🟠 Elastic


🟠 ElasticNet - Stock 444/975: TTMI.O

🟠 ElasticNet - Stock 445/975: CXW

🟠 ElasticNet - Stock 446/975: SSRM.O

🟠 ElasticNet - Stock 447/975: DIOD.O

🟠 ElasticNet - Stock 448/975: AEO

🟠 ElasticNet - Stock 449/975: ARWR.O

🟠 ElasticNet - Stock 450/975: WMK

🟠 ElasticNet - Stock 451/975: AGYS.O

🟠 ElasticNet - Stock 452/975: LGFa

🟠 ElasticNet - Stock 453/975: LGND.O

🟠 ElasticNet - Stock 454/975: HNI

🟠 ElasticNet - Stock 455/975: JBLU.O

🟠 ElasticNet - Stock 456/975: NEOG.O

🟠 ElasticNet - Stock 457/975: HE

🟠 ElasticNet - Stock 458/975: EXTR.O

🟠 ElasticNet - Stock 459/975: ACLS.O

🟠 ElasticNet - Stock 460/975: WERN.O

🟠 ElasticNet - Stock 461/975: LMAT.O

🟠 ElasticNet - Stock 462/975: CENX.O

🟠 ElasticNet - Stock 463/975: BKE

🟠 ElasticNet - Stock 464/975: CNMD.K

🟠 ElasticNet - Stock 465/975: OI

🟠 ElasticNet - Stock 466/975: CSGS.O

🟠 ElasticNet - Stock 467/975: RAMP.K

🟠 ElasticNet - Stock 468/975: IART.O

🟠 ElasticNet - Stock 469/975: ZD.O

🟠 ElasticNet - Stock 470/975: PBI

🟠 E


🟠 ElasticNet - Stock 659/975: DDD

🟠 ElasticNet - Stock 660/975: APPS.O

🟠 ElasticNet - Stock 661/975: MSB

🟠 ElasticNet - Stock 662/975: PNRG.O

🟠 ElasticNet - Stock 663/975: ACTG.O

🟠 ElasticNet - Stock 664/975: HVT

🟠 ElasticNet - Stock 665/975: ATNI.O

🟠 ElasticNet - Stock 666/975: MTW

🟠 ElasticNet - Stock 667/975: MATV.K

🟠 ElasticNet - Stock 668/975: SPOK.O

🟠 ElasticNet - Stock 669/975: SHYF.O

🟠 ElasticNet - Stock 670/975: NEWT.O

🟠 ElasticNet - Stock 671/975: CYRX.O

🟠 ElasticNet - Stock 672/975: NVEC.O

🟠 ElasticNet - Stock 673/975: EBS

🟠 ElasticNet - Stock 674/975: OIS

🟠 ElasticNet - Stock 675/975: RLGT.K

🟠 ElasticNet - Stock 676/975: BYON.K

🟠 ElasticNet - Stock 677/975: MVIS.O

🟠 ElasticNet - Stock 678/975: UIS

🟠 ElasticNet - Stock 679/975: CDZI.O

🟠 ElasticNet - Stock 680/975: VNDA.O

🟠 ElasticNet - Stock 681/975: NGS

🟠 ElasticNet - Stock 682/975: EGHT.O

🟠 ElasticNet - Stock 683/975: PAMT.O

🟠 ElasticNet - Stock 684/975: FNLC.O

🟠 ElasticNet - Stock 685/975: AEHR.


🟠 ElasticNet - Stock 872/975: ACCS.K

🟠 ElasticNet - Stock 873/975: AWRE.O

🟠 ElasticNet - Stock 874/975: AIFF.O

🟠 ElasticNet - Stock 875/975: SGRP.O

🟠 ElasticNet - Stock 876/975: NTIP.K

🟠 ElasticNet - Stock 877/975: VHC

🟠 ElasticNet - Stock 878/975: FTEK.O

🟠 ElasticNet - Stock 879/975: GROW.O

🟠 ElasticNet - Stock 880/975: IRD.O

🟠 ElasticNet - Stock 881/975: CMS_pb

🟠 ElasticNet - Stock 882/975: KTCC.O

🟠 ElasticNet - Stock 883/975: HSON.O

🟠 ElasticNet - Stock 884/975: AXDX.O

🟠 ElasticNet - Stock 885/975: HBIO.O

🟠 ElasticNet - Stock 886/975: NSYS.O

🟠 ElasticNet - Stock 887/975: OCC.O

🟠 ElasticNet - Stock 888/975: NTWK.O

🟠 ElasticNet - Stock 889/975: INTG.O

🟠 ElasticNet - Stock 890/975: DRRX.O

🟠 ElasticNet - Stock 891/975: IGC

🟠 ElasticNet - Stock 892/975: TRT

🟠 ElasticNet - Stock 893/975: ENZ

🟠 ElasticNet - Stock 894/975: GTIM.O

🟠 ElasticNet - Stock 895/975: TENX.O

🟠 ElasticNet - Stock 896/975: CKX

🟠 ElasticNet - Stock 897/975: CRIS.O

🟠 ElasticNet - Stock 898/975

### Hyperparameter Tuning - SVR

In [7]:
# Define the Fama-French factor columns
ff_5_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in five_ff_cleaned_df.columns if col not in ff_5_factors]

# Start global timer
start_all = time.time()

# ------------------ SVR ------------------

all_svr_results = []
best_svr_models = {}

# ------------------ Model Selection Loop ------------------
# Loop through each stock
for idx, stock in enumerate(stock_columns):
    print(f"\n🟠 SVR - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()
    # Subset and process
    df_subset = five_ff_cleaned_df[ff_5_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'Stock_Return', 'RF']].dropna()
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Excess_Return'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']])

    svr_results = []

    for C in svr_C_values:
        for epsilon in svr_epsilons:
            model = SVR(C=C, epsilon=epsilon)
            val_rmse_list = []

            for i in range(train_size, train_size + val_size - 3):
                train_end = month_periods[i - 1]
                val_start = month_periods[i]
                val_end = month_periods[i + 2]

                train_mask = df_subset['Date'].dt.to_period('M') <= train_end
                val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & (df_subset['Date'].dt.to_period('M') <= val_end)


                X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
                y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
                X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
                y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

                model.fit(X_train, y_train)
                rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
                val_rmse_list.append(rmse)

            svr_results.append({
                'Stock': stock,
                'C': C,
                'Epsilon': epsilon,
                'Avg_Validation_RMSE': np.mean(val_rmse_list),
                'Model': model
            })

    svr_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_svr_models[stock] = svr_results[0]['Model']
    all_svr_results.append(svr_results[0])

print(f"✅ SVR done for {stock} | Best C: {svr_results[0]['C']} | Epsilon: {svr_results[0]['Epsilon']}")


🟠 SVR - Stock 1/975: AAPL.O

🟠 SVR - Stock 2/975: NVDA.O

🟠 SVR - Stock 3/975: MSFT.O

🟠 SVR - Stock 4/975: AMZN.O

🟠 SVR - Stock 5/975: LLY

🟠 SVR - Stock 6/975: WMT

🟠 SVR - Stock 7/975: XOM

🟠 SVR - Stock 8/975: MA

🟠 SVR - Stock 9/975: UNH

🟠 SVR - Stock 10/975: ORCL.K

🟠 SVR - Stock 11/975: COST.O

🟠 SVR - Stock 12/975: NFLX.O

🟠 SVR - Stock 13/975: HD

🟠 SVR - Stock 14/975: CVX

🟠 SVR - Stock 15/975: CRM

🟠 SVR - Stock 16/975: CSCO.O

🟠 SVR - Stock 17/975: MRK

🟠 SVR - Stock 18/975: ABT

🟠 SVR - Stock 19/975: MCD

🟠 SVR - Stock 20/975: TMO

🟠 SVR - Stock 21/975: ISRG.O

🟠 SVR - Stock 22/975: RTX

🟠 SVR - Stock 23/975: QCOM.O

🟠 SVR - Stock 24/975: ADBE.O

🟠 SVR - Stock 25/975: AMGN.O

🟠 SVR - Stock 26/975: INTU.O

🟠 SVR - Stock 27/975: AMD.O

🟠 SVR - Stock 28/975: CAT

🟠 SVR - Stock 29/975: TXN.O

🟠 SVR - Stock 30/975: NEE

🟠 SVR - Stock 31/975: DHR

🟠 SVR - Stock 32/975: PFE

🟠 SVR - Stock 33/975: BLK

🟠 SVR - Stock 34/975: UNP

🟠 SVR - Stock 35/975: BSX

🟠 SVR - Stock 36/975: 


🟠 SVR - Stock 276/975: RLI

🟠 SVR - Stock 277/975: IVZ

🟠 SVR - Stock 278/975: ONTO.K

🟠 SVR - Stock 279/975: CWST.O

🟠 SVR - Stock 280/975: QRVO.O

🟠 SVR - Stock 281/975: AAON.O

🟠 SVR - Stock 282/975: ALK

🟠 SVR - Stock 283/975: UFPI.O

🟠 SVR - Stock 284/975: FLS

🟠 SVR - Stock 285/975: SNV

🟠 SVR - Stock 286/975: TGTX.O

🟠 SVR - Stock 287/975: TFX

🟠 SVR - Stock 288/975: CELH.O

🟠 SVR - Stock 289/975: EAT

🟠 SVR - Stock 290/975: PEGA.O

🟠 SVR - Stock 291/975: OSK

🟠 SVR - Stock 292/975: LNC

🟠 SVR - Stock 293/975: AWI

🟠 SVR - Stock 294/975: CHH

🟠 SVR - Stock 295/975: WEX

🟠 SVR - Stock 296/975: CORT.O

🟠 SVR - Stock 297/975: MSA

🟠 SVR - Stock 298/975: RMBS.O

🟠 SVR - Stock 299/975: FCN

🟠 SVR - Stock 300/975: BMI

🟠 SVR - Stock 301/975: MMSI.O

🟠 SVR - Stock 302/975: MKSI.O

🟠 SVR - Stock 303/975: CACC.O

🟠 SVR - Stock 304/975: BYD

🟠 SVR - Stock 305/975: DDS

🟠 SVR - Stock 306/975: IDCC.O

🟠 SVR - Stock 307/975: ACIW.O

🟠 SVR - Stock 308/975: USM

🟠 SVR - Stock 309/975: CROX.O



🟠 SVR - Stock 543/975: SBGI.O

🟠 SVR - Stock 544/975: GRC

🟠 SVR - Stock 545/975: AMSF.O

🟠 SVR - Stock 546/975: KFRC.K

🟠 SVR - Stock 547/975: BELFA.O

🟠 SVR - Stock 548/975: THRM.O

🟠 SVR - Stock 549/975: LQDT.O

🟠 SVR - Stock 550/975: MYGN.O

🟠 SVR - Stock 551/975: NSSC.O

🟠 SVR - Stock 552/975: UTL

🟠 SVR - Stock 553/975: WOLF.K

🟠 SVR - Stock 554/975: KSS

🟠 SVR - Stock 555/975: CBRL.O

🟠 SVR - Stock 556/975: DCO

🟠 SVR - Stock 557/975: ATRO.O

🟠 SVR - Stock 558/975: HSII.O

🟠 SVR - Stock 559/975: GIC

🟠 SVR - Stock 560/975: PDFS.O

🟠 SVR - Stock 561/975: SCSC.O

🟠 SVR - Stock 562/975: STAA.O

🟠 SVR - Stock 563/975: ASTE.O

🟠 SVR - Stock 564/975: CECO.O

🟠 SVR - Stock 565/975: ECPG.O

🟠 SVR - Stock 566/975: PRA

🟠 SVR - Stock 567/975: AVXL.O

🟠 SVR - Stock 568/975: COHU.O

🟠 SVR - Stock 569/975: AMSC.O

🟠 SVR - Stock 570/975: HCKT.O

🟠 SVR - Stock 571/975: AIOT.O

🟠 SVR - Stock 572/975: PRAA.O

🟠 SVR - Stock 573/975: AXGN.O

🟠 SVR - Stock 574/975: OSBC.O

🟠 SVR - Stock 575/975: U


🟠 SVR - Stock 805/975: HGBL.O

🟠 SVR - Stock 806/975: PETS.O

🟠 SVR - Stock 807/975: DIT

🟠 SVR - Stock 808/975: AXTI.O

🟠 SVR - Stock 809/975: FOSL.O

🟠 SVR - Stock 810/975: ALOT.O

🟠 SVR - Stock 811/975: HNNA.O

🟠 SVR - Stock 812/975: RRGB.O

🟠 SVR - Stock 813/975: UBCP.O

🟠 SVR - Stock 814/975: MNOV.O

🟠 SVR - Stock 815/975: INVE.O

🟠 SVR - Stock 816/975: SGA.O

🟠 SVR - Stock 817/975: ESP

🟠 SVR - Stock 818/975: OPTT.K

🟠 SVR - Stock 819/975: CODA.O

🟠 SVR - Stock 820/975: AUBN.O

🟠 SVR - Stock 821/975: INO.O

🟠 SVR - Stock 822/975: IOR

🟠 SVR - Stock 823/975: MLSS.K

🟠 SVR - Stock 824/975: AREN.K

🟠 SVR - Stock 825/975: VERU.O

🟠 SVR - Stock 826/975: ASYS.O

🟠 SVR - Stock 827/975: ARMP.K

🟠 SVR - Stock 828/975: ASRT.O

🟠 SVR - Stock 829/975: FKWL.O

🟠 SVR - Stock 830/975: DOMH.O

🟠 SVR - Stock 831/975: DLHC.O

🟠 SVR - Stock 832/975: PED

🟠 SVR - Stock 833/975: ALTS.O

🟠 SVR - Stock 834/975: CULP.K

🟠 SVR - Stock 835/975: SUP

🟠 SVR - Stock 836/975: CATO.K

🟠 SVR - Stock 837/975: C

### Hyperparameter Tuning - DecisionTreeRegressor

In [8]:
# Define the Fama-French factor columns
ff_5_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in five_ff_cleaned_df.columns if col not in ff_5_factors]

# Start global timer
start_all = time.time()

# ------------------ Decision Tree Regression ------------------

all_dt_results = []
best_dt_models = {}

# ------------------ Model Selection Loop ------------------
# Loop through each stock
for idx, stock in enumerate(stock_columns):
    print(f"\n🌲 Decision Tree - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()
    # Subset and process
    df_subset = five_ff_cleaned_df[ff_5_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'Stock_Return', 'RF']].dropna()
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Excess_Return'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']])
    
    dt_results = []

    for max_depth in tree_depths:
        model = DecisionTreeRegressor(max_depth=max_depth, random_state=random_seed)
        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end = month_periods[i - 1]
            val_start = month_periods[i]
            val_end = month_periods[i + 2]

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & \
                       (df_subset['Date'].dt.to_period('M') <= val_end)


            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
            val_rmse_list.append(rmse)

        dt_results.append({
            'Stock': stock,
            'Max_Depth': max_depth,
            'Avg_Validation_RMSE': np.mean(val_rmse_list),
            'Model': model
        })

    dt_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_dt_models[stock] = dt_results[0]['Model']
    all_dt_results.append(dt_results[0])

print(f"✅ Decision Tree done for {stock} | Best max_depth: {dt_results[0]['Max_Depth']}")


🌲 Decision Tree - Stock 1/975: AAPL.O

🌲 Decision Tree - Stock 2/975: NVDA.O

🌲 Decision Tree - Stock 3/975: MSFT.O

🌲 Decision Tree - Stock 4/975: AMZN.O

🌲 Decision Tree - Stock 5/975: LLY

🌲 Decision Tree - Stock 6/975: WMT

🌲 Decision Tree - Stock 7/975: XOM

🌲 Decision Tree - Stock 8/975: MA

🌲 Decision Tree - Stock 9/975: UNH

🌲 Decision Tree - Stock 10/975: ORCL.K

🌲 Decision Tree - Stock 11/975: COST.O

🌲 Decision Tree - Stock 12/975: NFLX.O

🌲 Decision Tree - Stock 13/975: HD

🌲 Decision Tree - Stock 14/975: CVX

🌲 Decision Tree - Stock 15/975: CRM

🌲 Decision Tree - Stock 16/975: CSCO.O

🌲 Decision Tree - Stock 17/975: MRK

🌲 Decision Tree - Stock 18/975: ABT

🌲 Decision Tree - Stock 19/975: MCD

🌲 Decision Tree - Stock 20/975: TMO

🌲 Decision Tree - Stock 21/975: ISRG.O

🌲 Decision Tree - Stock 22/975: RTX

🌲 Decision Tree - Stock 23/975: QCOM.O

🌲 Decision Tree - Stock 24/975: ADBE.O

🌲 Decision Tree - Stock 25/975: AMGN.O

🌲 Decision Tree - Stock 26/975: INTU.O

🌲 Decisio


🌲 Decision Tree - Stock 209/975: JEF

🌲 Decision Tree - Stock 210/975: COKE.O

🌲 Decision Tree - Stock 211/975: TXRH.O

🌲 Decision Tree - Stock 212/975: SWKS.O

🌲 Decision Tree - Stock 213/975: NBIX.O

🌲 Decision Tree - Stock 214/975: ITT

🌲 Decision Tree - Stock 215/975: VTRS.O

🌲 Decision Tree - Stock 216/975: SNX

🌲 Decision Tree - Stock 217/975: AIZ

🌲 Decision Tree - Stock 218/975: RBC

🌲 Decision Tree - Stock 219/975: OVV

🌲 Decision Tree - Stock 220/975: MANH.O

🌲 Decision Tree - Stock 221/975: EMN

🌲 Decision Tree - Stock 222/975: CCK

🌲 Decision Tree - Stock 223/975: TOL

🌲 Decision Tree - Stock 224/975: CLH

🌲 Decision Tree - Stock 225/975: GME

🌲 Decision Tree - Stock 226/975: EXEL.O

🌲 Decision Tree - Stock 227/975: RGLD.O

🌲 Decision Tree - Stock 228/975: COHR.K

🌲 Decision Tree - Stock 229/975: DOX.O

🌲 Decision Tree - Stock 230/975: IPG

🌲 Decision Tree - Stock 231/975: TECH.O

🌲 Decision Tree - Stock 232/975: ATR

🌲 Decision Tree - Stock 233/975: WBA.O

🌲 Decision Tree


🌲 Decision Tree - Stock 412/975: GEF

🌲 Decision Tree - Stock 413/975: GSAT.O

🌲 Decision Tree - Stock 414/975: TEX

🌲 Decision Tree - Stock 415/975: IPGP.O

🌲 Decision Tree - Stock 416/975: PENN.O

🌲 Decision Tree - Stock 417/975: JOE

🌲 Decision Tree - Stock 418/975: HURN.O

🌲 Decision Tree - Stock 419/975: JJSF.O

🌲 Decision Tree - Stock 420/975: GT.O

🌲 Decision Tree - Stock 421/975: SAM

🌲 Decision Tree - Stock 422/975: HP

🌲 Decision Tree - Stock 423/975: PSMT.O

🌲 Decision Tree - Stock 424/975: CPRX.O

🌲 Decision Tree - Stock 425/975: FORM.O

🌲 Decision Tree - Stock 426/975: SYNA.O

🌲 Decision Tree - Stock 427/975: QDEL.O

🌲 Decision Tree - Stock 428/975: VIAV.O

🌲 Decision Tree - Stock 429/975: VICR.O

🌲 Decision Tree - Stock 430/975: BFH

🌲 Decision Tree - Stock 431/975: PRGS.O

🌲 Decision Tree - Stock 432/975: TRN

🌲 Decision Tree - Stock 433/975: CAKE.O

🌲 Decision Tree - Stock 434/975: WLY

🌲 Decision Tree - Stock 435/975: PAR

🌲 Decision Tree - Stock 436/975: VSEC.O

🌲 De


🌲 Decision Tree - Stock 612/975: SCHL.O

🌲 Decision Tree - Stock 613/975: DJCO.O

🌲 Decision Tree - Stock 614/975: NPKI.K

🌲 Decision Tree - Stock 615/975: AXL

🌲 Decision Tree - Stock 616/975: CMCO.O

🌲 Decision Tree - Stock 617/975: SLP.O

🌲 Decision Tree - Stock 618/975: CRDb

🌲 Decision Tree - Stock 619/975: TWI

🌲 Decision Tree - Stock 620/975: HZO

🌲 Decision Tree - Stock 621/975: CLMB.O

🌲 Decision Tree - Stock 622/975: MLR

🌲 Decision Tree - Stock 623/975: CCRN.O

🌲 Decision Tree - Stock 624/975: YORW.O

🌲 Decision Tree - Stock 625/975: BBW

🌲 Decision Tree - Stock 626/975: VLGEA.O

🌲 Decision Tree - Stock 627/975: LGTY.O

🌲 Decision Tree - Stock 628/975: PBT

🌲 Decision Tree - Stock 629/975: MYE

🌲 Decision Tree - Stock 630/975: WNC

🌲 Decision Tree - Stock 631/975: GTN

🌲 Decision Tree - Stock 632/975: KELYA.O

🌲 Decision Tree - Stock 633/975: ALRS.O

🌲 Decision Tree - Stock 634/975: ALT.O

🌲 Decision Tree - Stock 635/975: EGY

🌲 Decision Tree - Stock 636/975: EYPT.O

🌲 Deci


🌲 Decision Tree - Stock 811/975: HNNA.O

🌲 Decision Tree - Stock 812/975: RRGB.O

🌲 Decision Tree - Stock 813/975: UBCP.O

🌲 Decision Tree - Stock 814/975: MNOV.O

🌲 Decision Tree - Stock 815/975: INVE.O

🌲 Decision Tree - Stock 816/975: SGA.O

🌲 Decision Tree - Stock 817/975: ESP

🌲 Decision Tree - Stock 818/975: OPTT.K

🌲 Decision Tree - Stock 819/975: CODA.O

🌲 Decision Tree - Stock 820/975: AUBN.O

🌲 Decision Tree - Stock 821/975: INO.O

🌲 Decision Tree - Stock 822/975: IOR

🌲 Decision Tree - Stock 823/975: MLSS.K

🌲 Decision Tree - Stock 824/975: AREN.K

🌲 Decision Tree - Stock 825/975: VERU.O

🌲 Decision Tree - Stock 826/975: ASYS.O

🌲 Decision Tree - Stock 827/975: ARMP.K

🌲 Decision Tree - Stock 828/975: ASRT.O

🌲 Decision Tree - Stock 829/975: FKWL.O

🌲 Decision Tree - Stock 830/975: DOMH.O

🌲 Decision Tree - Stock 831/975: DLHC.O

🌲 Decision Tree - Stock 832/975: PED

🌲 Decision Tree - Stock 833/975: ALTS.O

🌲 Decision Tree - Stock 834/975: CULP.K

🌲 Decision Tree - Stock 83

### Hyperparameter Tuning - XGBoost

In [9]:
# Define the Fama-French factor columns
ff_5_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in five_ff_cleaned_df.columns if col not in ff_5_factors]

# Start global timer
start_all = time.time()

# ------------------ XGBoost ------------------

all_xgb_results = []
best_xgb_models = {}

# ------------------ Model Selection Loop ------------------
# Loop through each stock
for idx, stock in enumerate(stock_columns):
    print(f"\n🟢 XGBoost - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()
    # Subset and process
    df_subset = five_ff_cleaned_df[ff_5_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'Stock_Return', 'RF']].dropna()
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Excess_Return'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']])
    
    xgb_results = []

    for n_estimators, lr in xgb_params:
        model = XGBRegressor(n_estimators=n_estimators, learning_rate=lr, random_state=random_seed)
        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end = month_periods[i - 1]
            val_start = month_periods[i]
            val_end = month_periods[i + 2]

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & (df_subset['Date'].dt.to_period('M') <= val_end)

            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
            val_rmse_list.append(rmse)

        xgb_results.append({
            'Stock': stock,
            'n_estimators': n_estimators,
            'learning_rate': lr,
            'Avg_Validation_RMSE': np.mean(val_rmse_list),
            'Model': model
        })

    xgb_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_xgb_models[stock] = xgb_results[0]['Model']
    all_xgb_results.append(xgb_results[0])

print(f"✅ XGBoost done for {stock} | Estimators: {xgb_results[0]['n_estimators']} | LR: {xgb_results[0]['learning_rate']}")


🟢 XGBoost - Stock 1/975: AAPL.O

🟢 XGBoost - Stock 2/975: NVDA.O

🟢 XGBoost - Stock 3/975: MSFT.O

🟢 XGBoost - Stock 4/975: AMZN.O

🟢 XGBoost - Stock 5/975: LLY

🟢 XGBoost - Stock 6/975: WMT

🟢 XGBoost - Stock 7/975: XOM

🟢 XGBoost - Stock 8/975: MA

🟢 XGBoost - Stock 9/975: UNH

🟢 XGBoost - Stock 10/975: ORCL.K

🟢 XGBoost - Stock 11/975: COST.O

🟢 XGBoost - Stock 12/975: NFLX.O

🟢 XGBoost - Stock 13/975: HD

🟢 XGBoost - Stock 14/975: CVX

🟢 XGBoost - Stock 15/975: CRM

🟢 XGBoost - Stock 16/975: CSCO.O

🟢 XGBoost - Stock 17/975: MRK

🟢 XGBoost - Stock 18/975: ABT

🟢 XGBoost - Stock 19/975: MCD

🟢 XGBoost - Stock 20/975: TMO

🟢 XGBoost - Stock 21/975: ISRG.O

🟢 XGBoost - Stock 22/975: RTX

🟢 XGBoost - Stock 23/975: QCOM.O

🟢 XGBoost - Stock 24/975: ADBE.O

🟢 XGBoost - Stock 25/975: AMGN.O

🟢 XGBoost - Stock 26/975: INTU.O

🟢 XGBoost - Stock 27/975: AMD.O

🟢 XGBoost - Stock 28/975: CAT

🟢 XGBoost - Stock 29/975: TXN.O

🟢 XGBoost - Stock 30/975: NEE

🟢 XGBoost - Stock 31/975: DHR

🟢 XGBo


🟢 XGBoost - Stock 245/975: LNW.O

🟢 XGBoost - Stock 246/975: CHE

🟢 XGBoost - Stock 247/975: CRL

🟢 XGBoost - Stock 248/975: DRS.O

🟢 XGBoost - Stock 249/975: RGEN.O

🟢 XGBoost - Stock 250/975: LSCC.O

🟢 XGBoost - Stock 251/975: EXAS.O

🟢 XGBoost - Stock 252/975: HAS.O

🟢 XGBoost - Stock 253/975: PARA.O

🟢 XGBoost - Stock 254/975: DCI

🟢 XGBoost - Stock 255/975: CHDN.O

🟢 XGBoost - Stock 256/975: MKTX.O

🟢 XGBoost - Stock 257/975: NYT

🟢 XGBoost - Stock 258/975: WCC

🟢 XGBoost - Stock 259/975: DLB

🟢 XGBoost - Stock 260/975: EVR

🟢 XGBoost - Stock 261/975: TTEK.O

🟢 XGBoost - Stock 262/975: RRX

🟢 XGBoost - Stock 263/975: SIRI.O

🟢 XGBoost - Stock 264/975: HALO.O

🟢 XGBoost - Stock 265/975: GAP

🟢 XGBoost - Stock 266/975: EXLS.O

🟢 XGBoost - Stock 267/975: ATI

🟢 XGBoost - Stock 268/975: TTC

🟢 XGBoost - Stock 269/975: ZION.O

🟢 XGBoost - Stock 270/975: BIO

🟢 XGBoost - Stock 271/975: MHK

🟢 XGBoost - Stock 272/975: APA.O

🟢 XGBoost - Stock 273/975: CVLT.O

🟢 XGBoost - Stock 274/975: 


🟢 XGBoost - Stock 482/975: MNKD.O

🟢 XGBoost - Stock 483/975: MCRI.O

🟢 XGBoost - Stock 484/975: APLD.O

🟢 XGBoost - Stock 485/975: SRCE.O

🟢 XGBoost - Stock 486/975: OMCL.O

🟢 XGBoost - Stock 487/975: INOD.O

🟢 XGBoost - Stock 488/975: FL

🟢 XGBoost - Stock 489/975: NTCT.O

🟢 XGBoost - Stock 490/975: EPC

🟢 XGBoost - Stock 491/975: ATSG.O

🟢 XGBoost - Stock 492/975: ADEA.O

🟢 XGBoost - Stock 493/975: ROG

🟢 XGBoost - Stock 494/975: NHC

🟢 XGBoost - Stock 495/975: LNN

🟢 XGBoost - Stock 496/975: PZZA.O

🟢 XGBoost - Stock 497/975: BHE

🟢 XGBoost - Stock 498/975: UPBD.O

🟢 XGBoost - Stock 499/975: ANIP.O

🟢 XGBoost - Stock 500/975: CLDX.O

🟢 XGBoost - Stock 501/975: VRNT.O

🟢 XGBoost - Stock 502/975: PLAB.O

🟢 XGBoost - Stock 503/975: VYX

🟢 XGBoost - Stock 504/975: DXPE.O

🟢 XGBoost - Stock 505/975: ATEC.O

🟢 XGBoost - Stock 506/975: NVAX.O

🟢 XGBoost - Stock 507/975: CTS

🟢 XGBoost - Stock 508/975: VSAT.O

🟢 XGBoost - Stock 509/975: NEO.O

🟢 XGBoost - Stock 510/975: WT

🟢 XGBoost - St


🟢 XGBoost - Stock 715/975: TSSI.O

🟢 XGBoost - Stock 716/975: LFVN.O

🟢 XGBoost - Stock 717/975: ESCA.O

🟢 XGBoost - Stock 718/975: SGMO.O

🟢 XGBoost - Stock 719/975: MPAA.O

🟢 XGBoost - Stock 720/975: DENN.O

🟢 XGBoost - Stock 721/975: FXNC.O

🟢 XGBoost - Stock 722/975: SWKH.O

🟢 XGBoost - Stock 723/975: UTMD.O

🟢 XGBoost - Stock 724/975: LAKE.O

🟢 XGBoost - Stock 725/975: GENC.K

🟢 XGBoost - Stock 726/975: LTBR.O

🟢 XGBoost - Stock 727/975: HQI.O

🟢 XGBoost - Stock 728/975: STRT.O

🟢 XGBoost - Stock 729/975: RELL.O

🟢 XGBoost - Stock 730/975: EVC

🟢 XGBoost - Stock 731/975: FENC.O

🟢 XGBoost - Stock 732/975: SMID.O

🟢 XGBoost - Stock 733/975: CATX.K

🟢 XGBoost - Stock 734/975: TBI

🟢 XGBoost - Stock 735/975: EPM

🟢 XGBoost - Stock 736/975: VOXX.O

🟢 XGBoost - Stock 737/975: DBI

🟢 XGBoost - Stock 738/975: NKTR.O

🟢 XGBoost - Stock 739/975: PLX

🟢 XGBoost - Stock 740/975: TTEC.O

🟢 XGBoost - Stock 741/975: PDEX.O

🟢 XGBoost - Stock 742/975: VIRC.O

🟢 XGBoost - Stock 743/975: ESOA.O




🟢 XGBoost - Stock 946/975: AIMD.O

🟢 XGBoost - Stock 947/975: AIM

🟢 XGBoost - Stock 948/975: STRR.O

🟢 XGBoost - Stock 949/975: SSY

🟢 XGBoost - Stock 950/975: GTBP.O

🟢 XGBoost - Stock 951/975: SGMA.O

🟢 XGBoost - Stock 952/975: OGEN.K

🟢 XGBoost - Stock 953/975: RIME.O

🟢 XGBoost - Stock 954/975: SNGX.O

🟢 XGBoost - Stock 955/975: SNOA.O

🟢 XGBoost - Stock 956/975: AEMD.O

🟢 XGBoost - Stock 957/975: GBR

🟢 XGBoost - Stock 958/975: KNW

🟢 XGBoost - Stock 959/975: BKYI.O

🟢 XGBoost - Stock 960/975: UUU

🟢 XGBoost - Stock 961/975: PSTV.O

🟢 XGBoost - Stock 962/975: FORD.O

🟢 XGBoost - Stock 963/975: AYRO.O

🟢 XGBoost - Stock 964/975: TTNP.O

🟢 XGBoost - Stock 965/975: TOVX.K

🟢 XGBoost - Stock 966/975: PRSO.O

🟢 XGBoost - Stock 967/975: PALI.O

🟢 XGBoost - Stock 968/975: APDN.O

🟢 XGBoost - Stock 969/975: SBET.O

🟢 XGBoost - Stock 970/975: ASTI.O

🟢 XGBoost - Stock 971/975: AWH.O

🟢 XGBoost - Stock 972/975: SUNE.O

🟢 XGBoost - Stock 973/975: TCRT.O

🟢 XGBoost - Stock 974/975: WINT.O



### Output the best tuned config for each model for each stock

In [10]:
# Step 1: Build rows from each model's best result list
rows = []

for stock in stock_columns:
    row = {'Stock': stock}

    # Ridge
    ridge = next((r for r in all_ridge_results if r['Stock'] == stock), None)
    if ridge:
        row['Ridge'] = f"α={ridge['Alpha']}"

    # Lasso
    lasso = next((r for r in all_lasso_results if r['Stock'] == stock), None)
    if lasso:
        row['Lasso'] = f"α={lasso['Alpha']}"

    # ElasticNet
    elastic = next((r for r in all_elastic_results if r['Stock'] == stock), None)
    if elastic:
        row['ElasticNet'] = f"α={elastic['Alpha']}, l1={elastic['L1_Ratio']}"

    # SVR
    svr = next((r for r in all_svr_results if r['Stock'] == stock), None)
    if svr:
        row['SVR'] = f"C={svr['C']}, ε={svr['Epsilon']}"

    # Decision Tree
    dt = next((r for r in all_dt_results if r['Stock'] == stock), None)
    if dt:
        row['DecisionTree'] = f"depth={dt['Max_Depth']}"
        
    # XGBoost
    xgb = next((r for r in all_xgb_results if r['Stock'] == stock), None)
    if xgb:
        row['XGBoost'] = f"est={xgb['n_estimators']}, lr={xgb['learning_rate']}"

    rows.append(row)

# Step 2: Create DataFrame
model_summary_df = pd.DataFrame(rows)
# Define output path
excel_path = 'output/models/5ff_tuned_para.xlsx'

# Export to Excel
model_summary_df.to_excel(excel_path, index=False)

PermissionError: [Errno 13] Permission denied: 'output/models/5ff_tuned_para.xlsx'